In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch


In [37]:
# Substitua pelo caminho real do seu dataset CSV
df = pd.read_csv("sentiment_tweets3.csv")
df.columns=['Indice','Texto','IndicadorDepressao']  # Esperado: colunas 'text' e 'label'
df = df.drop('Indice', axis=1)


In [38]:
import re
# Remover URLs
def remove_URL(text):
    return re.sub(r'http\S+|www\S' , '',text)

df['Texto'] = df['Texto'].apply(remove_URL)
df.head(10)

,Texto,IndicadorDepressao
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga,0
2,@comeagainjen -,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
5,so sleepy. good times tonight though,0
6,"@SilkCharm re: #nbn as someone already said, d...",0
7,23 or 24ï¿½C possible today. Nice,0
8,nite twitterville workout in the am -ciao,0
9,"@daNanner Night, darlin'! Sweet dreams to you",0


In [39]:
#importa o dicionário de abreviações e gírias comuns de chat/mensagens
from chatWords import chat_words

#Substitui abreviações e gírias comuns de chat/mensagens por suas formas completas.
def chat_word(text):
    for a,b in chat_words.items():
        text=text.replace(a,b)
    return text

df['Texto']=df['Texto'].apply(chat_word)


In [40]:
from nltk.corpus import stopwords
#Remover stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['Texto']=df['Texto'].apply(remove_stopwords)
df.head(10)

,Texto,IndicadorDepressao
0,"real good moment. missssssssss much,",0
1,reading manga,0
2,@comeagainjen -,0
3,@lapcat Need send 'em accountant tomorrow. Odd...,0
4,ADD MYSPACE!!! myspace.com/LookThunder,0
5,sleepy. good times tonight though,0
6,"@SilkCharm re: #nbn someone already said, fibe...",0
7,23 24ï¿½C possible today. Nice,0
8,nite twitterville workout -ciao,0
9,"@daNanner Night, darlin'! Sweet dreams",0


In [41]:
import emoji
# Remover emojis
def remove_ej(text):
    return emoji.demojize(text)

df['Texto'] = df['Texto'].apply(remove_ej)
df.head(10)

,Texto,IndicadorDepressao
0,"real good moment. missssssssss much,",0
1,reading manga,0
2,@comeagainjen -,0
3,@lapcat Need send 'em accountant tomorrow. Odd...,0
4,ADD MYSPACE!!! myspace.com/LookThunder,0
5,sleepy. good times tonight though,0
6,"@SilkCharm re: #nbn someone already said, fibe...",0
7,23 24ï¿½C possible today. Nice,0
8,nite twitterville workout -ciao,0
9,"@daNanner Night, darlin'! Sweet dreams",0


In [42]:
# Remove todos os @ do texto
df["Texto"] = df["Texto"].str.replace(r"[@#/.com]", "", regex=True)
df.head(10)

,Texto,IndicadorDepressao
0,"real gd ent issssssssss uh,",0
1,reading anga,0
2,eagainjen -,0
3,"lapat Need send 'e auntant trrw Oddly, even re...",0
4,ADD MYSPACE!!! yspaeLkThunder,0
5,sleepy gd ties tnight thugh,0
6,"SilkChar re: nbn sene already said, fiber he e...",0
7,23 24ï¿½C pssible tday Nie,0
8,nite twitterville wrkut -ia,0
9,"daNanner Night, darlin'! Sweet dreas",0


In [43]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Texto'].tolist(), df['IndicadorDepressao'].tolist(), test_size=0.2, random_state=42
)

In [44]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


In [45]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})


In [46]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
import os

os.makedirs("./logs", exist_ok=True)
os.makedirs("./meu_modelo", exist_ok=True) 

In [48]:
from transformers import Trainer
import torch

class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Aplicar os pesos das classes
        if self.class_weights is not None:
            # Mover os pesos para o mesmo device do modelo
            class_weights = torch.tensor(self.class_weights).to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
            
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [49]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # pega a classe com maior probabilidade
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [51]:
# Calcular os pesos das classes
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assumindo que train_labels são seus rótulos de treinamento
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
).astype(np.float32)

# Configurar os argumentos de treinamento normalmente
training_args = TrainingArguments(
    output_dir="./meu_modelo",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=600,
    learning_rate=1e-5,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Usar o CustomTrainer com os pesos das classes
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    class_weights=class_weights  # Passar os pesos aqui
)

# Iniciar o treinamento
trainer.train()


c:\Users\pedro\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/516 [01:59<?, ?it/s]
                                                
  2%|▏         | 10/516 [00:37<31:12,  3.70s/it]

{'loss': 0.696, 'grad_norm': 0.9285956025123596, 'learning_rate': 1.6666666666666668e-07, 'epoch': 0.04}


                                                
  4%|▍         | 20/516 [01:15<32:01,  3.87s/it]

{'loss': 0.7003, 'grad_norm': 1.1790043115615845, 'learning_rate': 3.3333333333333335e-07, 'epoch': 0.08}


                                                
  6%|▌         | 30/516 [01:54<31:12,  3.85s/it]

...

100%|██████████| 516/516 [35:02<00:00,  3.68s/it]
































































                                                 

                                         
100%|██████████| 516/516 [36:20<00:00,  3.68s/it]



{'eval_loss': 0.013572542928159237, 'eval_accuracy': 0.9975763451284537, 'eval_precision': 0.9955357142857143, 'eval_recall': 0.9933184855233853, 'eval_f1': 0.9944258639910813, 'eval_runtime': 78.0052, 'eval_samples_per_second': 26.447, 'eval_steps_per_second': 0.833, 'epoch': 2.0}


                                                 
100%|██████████| 516/516 [36:23<00:00,  4.23s/it]

{'train_runtime': 2183.0522, 'train_samples_per_second': 7.559, 'train_steps_per_second': 0.236, 'train_loss': 0.22816169376362316, 'epoch': 2.0}


TrainOutput(global_step=516, training_loss=0.22816169376362316, metrics={'train_runtime': 2183.0522, 'train_samples_per_second': 7.559, 'train_steps_per_second': 0.236, 'total_flos': 546494253155328.0, 'train_loss': 0.22816169376362316, 'epoch': 2.0})

In [52]:
trainer.evaluate()


100%|██████████| 65/65 [01:24<00:00,  1.30s/it]


{'eval_loss': 0.013572542928159237,
 'eval_accuracy': 0.9975763451284537,
 'eval_precision': 0.9955357142857143,
 'eval_recall': 0.9933184855233853,
 'eval_f1': 0.9944258639910813,
 'eval_runtime': 85.8929,
 'eval_samples_per_second': 24.018,
 'eval_steps_per_second': 0.757,
 'epoch': 2.0}

In [53]:
model.save_pretrained("./meu_modelo_distilbert")
tokenizer.save_pretrained("./meu_modelo_distilbert")


('./meu_modelo_distilbert\\tokenizer_config.json',
 './meu_modelo_distilbert\\special_tokens_map.json',
 './meu_modelo_distilbert\\vocab.txt',
 './meu_modelo_distilbert\\added_tokens.json',
 './meu_modelo_distilbert\\tokenizer.json')

In [56]:
from transformers import pipeline

# pipeline de classificação com modelo salvo
classificador = pipeline("text-classification", model="./meu_modelo_distilbert", tokenizer="./meu_modelo_distilbert")

# Exemplo de uso:
classificador("Major signs of depression. Seriously, I hope you are seeking help.")

[{'label': 'LABEL_1', 'score': 0.9482271075248718}]